# Building a seq2seq model for machine translation.

### 1.1. Load and clean text

In [1]:
import re
import string
from unicodedata import normalize
import numpy

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs

def clean_data(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return numpy.array(cleaned)

In [2]:
# e.g., filename = 'Data/deu.txt'
filename = 'D:\GIT-REPOS\ML-DL_Algos\RNN\data\TranslationData\spa.txt'

# e.g., n_train = 20000
n_train = 20000

<>:2: SyntaxWarning: invalid escape sequence '\G'
<>:2: SyntaxWarning: invalid escape sequence '\G'
C:\Users\abdul\AppData\Local\Temp\ipykernel_20320\1854643258.py:2: SyntaxWarning: invalid escape sequence '\G'
  filename = 'D:\GIT-REPOS\ML-DL_Algos\RNN\data\TranslationData\spa.txt'


In [3]:
# load dataset
doc = load_doc(filename)

# split into Language1-Language2 pairs
pairs = to_pairs(doc)

# clean sentences
clean_pairs = clean_data(pairs)[0:n_train, :]

In [4]:
for i in range(3000, 3010):
    print('[' + clean_pairs[i, 0] + '] => [' + clean_pairs[i, 1] + ']')

[we are here] => [estamos aqui]
[we ate eggs] => [hemos comido huevos]
[we ate eggs] => [comimos huevos]
[we broke up] => [nos separamos]
[we broke up] => [lo dejamos]
[we broke up] => [rompimos]
[we can help] => [podemos ayudar]
[we can help] => [nosotros podemos ayudar]
[we can meet] => [podemos encontrarnos]
[we can meet] => [podemos vernos]


In [13]:
input_texts = clean_pairs[:, 0]
target_texts = ['\t' + text + '\n' for text in clean_pairs[:, 1]]

print('Length of input_texts:  ' + str(input_texts.shape))
print('Length of target_texts: ' + str(len(target_texts)))

Length of input_texts:  (20000,)
Length of target_texts: 20000


In [6]:
max_encoder_seq_length = max(len(line) for line in input_texts)
max_decoder_seq_length = max(len(line) for line in target_texts)

print('max length of input  sentences: %d' % (max_encoder_seq_length))
print('max length of target sentences: %d' % (max_decoder_seq_length))

max length of input  sentences: 18
max length of target sentences: 48


**Remark:** To this end, we have two lists of sentences: input_texts and target_texts

## 2. Text processing

### 2.1. Convert texts to sequences

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# encode and pad sequences
def text2sequences(max_len, lines):
    tokenizer = Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(lines)
    seqs = tokenizer.texts_to_sequences(lines)
    seqs_pad = pad_sequences(seqs, maxlen=max_len, padding='post')
    return seqs_pad, tokenizer.word_index


encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length,
                                                      input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length,
                                                       target_texts)

print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

shape of encoder_input_seq: (20000, 18)
shape of input_token_index: 27
shape of decoder_input_seq: (20000, 48)
shape of target_token_index: 29


In [14]:
num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

print('num_encoder_tokens: ' + str(num_encoder_tokens))
print('num_decoder_tokens: ' + str(num_decoder_tokens))

num_encoder_tokens: 28
num_decoder_tokens: 30


**Remark:** To this end, the input language and target language texts are converted to 2 matrices.

- Their number of rows are both n_train.
- Their number of columns are respective max_encoder_seq_length and max_decoder_seq_length.

The followings print a sentence and its representation as a sequence.

In [15]:
target_texts[100]

'\tentendiste\n'

In [16]:
decoder_input_seq[100, :]

array([ 6,  2,  9,  8,  2,  9, 15, 11,  5,  8,  2,  7,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

## 2.2. One-hot encode

In [17]:
from tensorflow.keras.utils import to_categorical

# one hot encode target sequence
def onehot_encode(sequences, max_len, vocab_size):
    n = len(sequences)
    data = numpy.zeros((n, max_len, vocab_size))
    for i in range(n):
        data[i, :, :] = to_categorical(sequences[i], num_classes=vocab_size)
    return data

encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq,
                                    max_decoder_seq_length,
                                    num_decoder_tokens)

print(encoder_input_data.shape)
print(decoder_input_data.shape)

(20000, 18, 28)
(20000, 48, 30)


## 3. Build the networks (for training)
 Bidirectional LSTM model

### 3.1. Encoder network
- Input:  one-hot encode of the input language

- Return:

    -- output (all the hidden states   $h_1, \cdots , h_t$) are always discarded
    
    -- the final hidden state  $h_t$
    
    -- the final conveyor belt $c_t$

In [18]:
from tensorflow.keras.layers import Input, LSTM , Bidirectional, Concatenate
from tensorflow.keras.models import Model

latent_dim = 256

# inputs of the encoder network
encoder_inputs = Input(shape=(None, num_encoder_tokens),
                       name='encoder_inputs')


# Build BiLSTM encoder
encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True, dropout=0.5, name='encoder_bilstm'))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

# Merge forward and backward states
state_h = Concatenate()([forward_h, backward_h])   # Final hidden state
state_c = Concatenate()([forward_c, backward_c])   # Final conveyor belt (cell state)

# set the LSTM layer
# encoder_lstm = LSTM(latent_dim, return_state=True,
#                     dropout=0.5, name='encoder_lstm')
# _, state_h, state_c = encoder_lstm(encoder_inputs)

# build the encoder network model
encoder_model = Model(inputs=encoder_inputs,
                      outputs=[state_h, state_c],
                      name='encoder')

In [19]:
encoder_model.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None, 28)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ [(None, 512),     │    583,680 │ encoder_inputs[0… │
│ (Bidirectional)     │ (None, 256),      │            │                   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 512)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 512)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 583,680 (2.23 MB)

 Trainable params: 583,680 (2.23 MB)

 Non-trainable params: 0 (0.00 B)

### 3.2. Decoder network

- Inputs:  

    -- one-hot encode of the target language
    
    -- The initial hidden state $h_t$
    
    -- The initial conveyor belt $c_t$

- Return:

    -- output (all the hidden states) $h_1, \cdots , h_t$

    -- the final hidden state  $h_t$ (discarded in the training and used in the prediction)
    
    -- the final conveyor belt $c_t$ (discarded in the training and used in the prediction)

In [20]:
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

# inputs of the decoder network
decoder_input_h = Input(shape=(latent_dim * 2,), name='decoder_input_h')
decoder_input_c = Input(shape=(latent_dim * 2,), name='decoder_input_c')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# set the LSTM layer
decoder_lstm = LSTM(latent_dim* 2, return_sequences=True,
                    return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_input_x,
                                                      initial_state=[decoder_input_h, decoder_input_c])

# set the dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# build the decoder network model
decoder_model = Model(inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
                      outputs=[decoder_outputs, state_h, state_c],
                      name='decoder')

In [21]:
decoder_model.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ decoder_input_x     │ (None, None, 30)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input_h     │ (None, 512)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input_c     │ (None, 512)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │  1,112,064 │ decoder_input_x[… │
│                     │ 512), (None,      │            │ decoder_input_h[… │
│                     │ 512), (None,      │            │ decoder_input_c[… │
│                     │ 512)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, None, 30)  │     15,390 │ decoder_lstm[0][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,127,454 (4.30 MB)

 Trainable params: 1,127,454 (4.30 MB)

 Non-trainable params: 0 (0.00 B)

### 3.3. Connect the encoder and decoder

In [22]:
# input layers
encoder_input_x = Input(shape=(None, num_encoder_tokens), name='encoder_input_x')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# connect encoder to decoder
encoder_final_states = encoder_model([encoder_input_x])
decoder_lstm_output, _, _ = decoder_lstm(decoder_input_x, initial_state=encoder_final_states)
decoder_pred = decoder_dense(decoder_lstm_output)

model = Model(inputs=[encoder_input_x, decoder_input_x],
              outputs=decoder_pred,
              name='model_training')

In [23]:
model.summary()

Model: "model_training"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input_x     │ (None, None, 28)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input_x     │ (None, None, 30)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder             │ [(None, 512),     │    583,680 │ encoder_input_x[… │
│ (Functional)        │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │  1,112,064 │ decoder_input_x[… │
│                     │ 512), (None,      │            │ encoder[0][0],    │
│                     │ 512), (None,      │            │ encoder[0][1]     │
│                     │ 512)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, None, 30)  │     15,390 │ decoder_lstm[1][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,711,134 (6.53 MB)

 Trainable params: 1,711,134 (6.53 MB)

 Non-trainable params: 0 (0.00 B)

### 3.4. Fit the model on the bilingual dataset

- encoder_input_data: one-hot encode of the input language

- decoder_input_data: one-hot encode of the input language

- decoder_target_data: labels (left shift of decoder_input_data)

- tune the hyper-parameters

- stop when the validation loss stop decreasing.

In [24]:
print('shape of encoder_input_data' + str(encoder_input_data.shape))
print('shape of decoder_input_data' + str(decoder_input_data.shape))
print('shape of decoder_target_data' + str(decoder_target_data.shape))

shape of encoder_input_data(20000, 18, 28)
shape of decoder_input_data(20000, 48, 30)
shape of decoder_target_data(20000, 48, 30)


In [25]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit([encoder_input_data, decoder_input_data],  # training data
          decoder_target_data,                       # labels (left shift of the target sequences)
          batch_size=64, epochs=50, validation_split=0.2)

model.save('seq2seq.h5')

Epoch 1/50


c:\Users\abdul\anaconda3\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: encoder_inputs
Received: inputs=['Tensor(shape=(64, 18, 28))']
  warnings.warn(msg)


250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step - loss: 1.3027

c:\Users\abdul\anaconda3\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: encoder_inputs
Received: inputs=['Tensor(shape=(None, 18, 28))']
  warnings.warn(msg)


250/250 ━━━━━━━━━━━━━━━━━━━━ 75s 289ms/step - loss: 1.3020 - val_loss: 1.1824
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 69s 277ms/step - loss: 0.9639 - val_loss: 1.0135
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 70s 282ms/step - loss: 0.8791 - val_loss: 0.9156
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 72s 289ms/step - loss: 0.8395 - val_loss: 0.8906
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 67s 267ms/step - loss: 0.8269 - val_loss: 0.8528
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 77s 308ms/step - loss: 0.8099 - val_loss: 0.8366
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 70s 280ms/step - loss: 0.7979 - val_loss: 0.8235
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 66s 262ms/step - loss: 0.7854 - val_loss: 0.8065
Epoch 9/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 64s 257ms/step - loss: 0.7763 - val_loss: 0.7913
Epoch 10/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 61s 242ms/step - loss: 0.7683 - val_loss: 0.7826
Epoch 11/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 60s 239ms/step - loss: 0.7625 - val_loss: 0.7691
Epoch 12/50
250/250 ━━━━━━━━━━

## 4. Make predictions

### 4.1. Translate English to XXX

1. Encoder read a sentence (source language) and output its final states, $h_t$ and $c_t$.
2. Take the [star] sign "\t" and the final state $h_t$ and $c_t$ as input and run the decoder.
3. Get the new states and predicted probability distribution.
4. sample a char from the predicted probability distribution
5. take the sampled char and the new states as input and repeat the process (stop if reach the [stop] sign "\n").

In [26]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [53]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = numpy.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # this line of code is greedy selection
        # try to use multinomial sampling instead (with temperature)
        sampled_token_index = numpy.argmax(output_tokens[0, -1, :])

        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = numpy.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

In [28]:
for seq_index in range(2100, 2120):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('English:       ', input_texts[seq_index])
    print('Spanish (true): ', target_texts[seq_index][1:-1])
    print('Spanish (pred): ', decoded_sentence[0:-1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 402ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
-
English:        come see me
Spanish (true):  venid a verme
Spanish (pred):  ven a aqui
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/st

### 4.2. Translate an English sentence to the target language 

1. Tokenization
2. One-hot encode
3. Translate

In [29]:
import numpy as np

input_sentence = 'I love you'

input_sequence = [input_token_index.get(char, 0) for char in input_sentence]

input_x = np.zeros((1, len(input_sequence), num_encoder_tokens), dtype='float32')
for t, idx in enumerate(input_sequence):
    input_x[0, t, idx] = 1.0

translated_sentence = decode_sequence(input_x)

print('source sentence is: ' + input_sentence)
print('translated sentence is: ' + translated_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 398ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
source sentence is: I love you
translated sentence is: me gusta el coche



# 5. Evaluate the translation using BLEU score

### 5.1. Partition the dataset to training, validation, and test. Build new token index.

In [ ]:
def get_clean_target_list(target_test_texts):
    result = []
    for i in target_test_texts:
        result.append('\t' + i + '\n')
    return result

In [ ]:
clean_pairs = clean_data(pairs)[0:50000, :]
input_texts = clean_pairs[:, 0]
target_texts = get_clean_target_list(clean_pairs[:, 1])

print('Length of input_texts:  ' + str(input_texts.shape))
print('Length of target_texts: ' + str(input_texts.shape))

In [ ]:
print(len(target_texts))
print(input_texts.shape)
print(target_texts[:10])

50000
(50000,)
['\tve\n', '\tvete\n', '\tvaya\n', '\tvayase\n', '\thola\n', '\tcorre\n', '\tcorran\n', '\thuye\n', '\tcorra\n', '\tcorred\n']


In [33]:
max_encoder_seq_length = max(len(line) for line in input_texts)
max_decoder_seq_length = max(len(line) for line in target_texts)

print('max length of input  sentences: %d' % (max_encoder_seq_length))
print('max length of target sentences: %d' % (max_decoder_seq_length))

max length of input  sentences: 24
max length of target sentences: 68


In [34]:
encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length, 
                                                      input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length, 
                                                       target_texts)

print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

shape of encoder_input_seq: (50000, 24)
shape of input_token_index: 27
shape of decoder_input_seq: (50000, 68)
shape of target_token_index: 29


In [35]:
num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

print('num_encoder_tokens: ' + str(num_encoder_tokens))
print('num_decoder_tokens: ' + str(num_decoder_tokens))

num_encoder_tokens: 28
num_decoder_tokens: 30


In [36]:
encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

print(encoder_input_data.shape)
print(decoder_input_data.shape)

(50000, 24, 28)
(50000, 68, 30)


In [37]:
rand_indices = numpy.random.permutation(50000)
train_indices = rand_indices[0:40000]
valid_indices = rand_indices[40000:45000]
test_indices = rand_indices[45000:50000]


encoder_input_data_train = encoder_input_data[train_indices,:]
encoder_input_data_valid = encoder_input_data[valid_indices,:]
encoder_input_data_test = encoder_input_data[test_indices,:] 

decoder_input_data_train = decoder_input_data[train_indices,:]
decoder_input_data_valid = decoder_input_data[valid_indices,:]
decoder_input_data_test = decoder_input_data[test_indices,:] 


decoder_target_data_train = decoder_target_data[train_indices,:]
decoder_target_data_valid = decoder_target_data[valid_indices,:]
decoder_target_data_test = decoder_target_data[test_indices,:] 

In [38]:
print(decoder_input_data_valid.shape)
print(decoder_target_data_valid.shape)
print(decoder_target_data_test.shape)
print(decoder_target_data_train.shape)
print(decoder_input_data_train.shape)
print(decoder_input_data_test.shape)
print(encoder_input_data_train.shape)
print(encoder_input_data_test.shape)
print(encoder_input_data_valid.shape)

(5000, 68, 30)
(5000, 68, 30)
(5000, 68, 30)
(40000, 68, 30)
(40000, 68, 30)
(5000, 68, 30)
(40000, 24, 28)
(5000, 24, 28)
(5000, 24, 28)


### 5.2 Retrai previous Bidirectional LSTM model with training and validation data and tune the parameters (learning rate, optimizer, etc) based on validation score.

1. Use the model structure in section 3 to train a new model with new training and validation datasets.
2. Based on validation BLEU score or loss to tune the parameters.

In [66]:
from tensorflow.keras.optimizers import RMSprop, Adam

optimizer = Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer, loss="categorical_crossentropy")

model.fit([encoder_input_data_train, decoder_input_data_train], decoder_target_data_train, batch_size=256, epochs=50, 
          validation_data=([encoder_input_data_valid, decoder_input_data_valid], decoder_target_data_valid))

model.save('seq2seq_tune.h5')

Epoch 1/50


c:\Users\abdul\anaconda3\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: encoder_inputs
Received: inputs=['Tensor(shape=(None, 24, 28))']
  warnings.warn(msg)


157/157 ━━━━━━━━━━━━━━━━━━━━ 155s 959ms/step - loss: 0.4725 - val_loss: 0.3422
Epoch 2/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 144s 919ms/step - loss: 0.4722 - val_loss: 0.3415
Epoch 3/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 171s 1s/step - loss: 0.4679 - val_loss: 0.3407
Epoch 4/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 154s 983ms/step - loss: 0.4665 - val_loss: 0.3398
Epoch 5/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 165s 1s/step - loss: 0.4673 - val_loss: 0.3393
Epoch 6/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 155s 989ms/step - loss: 0.4685 - val_loss: 0.3392
Epoch 7/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 155s 987ms/step - loss: 0.4667 - val_loss: 0.3393
Epoch 8/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 157s 998ms/step - loss: 0.4654 - val_loss: 0.3394
Epoch 9/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 133s 848ms/step - loss: 0.4644 - val_loss: 0.3390
Epoch 10/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 166s 1s/step - loss: 0.4657 - val_loss: 0.3385
Epoch 11/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 159s 1s/step - loss: 0.4613 - val_loss: 0.3372
Epoch 12/50
157/157 ━━━━━━━━━━━

In [67]:
input_test_texts = input_texts[test_indices]
target_texts = numpy.array(target_texts)
target_test_texts = target_texts[test_indices]

predicted_sentences = []

# for seq_index in range(2100, 2120):
for seq_index in range(2100, 2120):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    predicted_sentences.append(decoded_sentence[0:-1])
    print('-')
    print('English:       ', input_test_texts[seq_index])
    print('Spanish (true): ', target_test_texts[seq_index][1:-1])
    print('Spanish (pred): ', decoded_sentence[0:-1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
-
English:        he walks to school
Spanish (true):  el va a la escuela caminando
Spanish (pred):  el salia del carcio
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━

### 5.3 Evaluate the BLEU score using the test set. 

In [74]:
def get_target_list_char_level(texts):
    result = []
    for i in texts:
        result.append(list(i.strip()))
    return result

In [75]:
target_list = get_target_list_char_level(target_test_texts[2100:2120])
predicted_list = get_target_list_char_level(predicted_sentences)

In [76]:
# Compute the BLEU score
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

smoothie = SmoothingFunction().method4

references = [[ref] for ref in target_list]  # corpus_bleu expects list of list of references
candidates = predicted_list
bleu = corpus_bleu(references, candidates, smoothing_function=smoothie)
print(bleu)

0.33696543664604084
